<a href="https://colab.research.google.com/github/PundirShivam/Portfolio/blob/master/s%26p500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import bs4

In [ ]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [ ]:
# getting page
response = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [ ]:
# soup of response page
soup = BeautifulSoup(response)

In [ ]:
# function to extract href links 
def get_href_links(soup):
  """
  The functionr returns a dictonary element
  with company_name:href link
  Input need to be the soup of the S&P500 page
  """
  # finding out the table element
  table = soup.find('table')
  # creating an list element for storage
  company_links={}
  # for each tr in table skipping the header column
  for tr in table.find_all('tr')[1:]:
    company_name= tr.find_all('a')[1].text
    href_link = str(tr.find_all('a')[1])
    if 'href' in href_link.split()[1]:
      href_link = href_link.split()[1].replace('href="',"").replace('"','')
    else:
      href_link = href_link.split()[2].replace('href="',"").replace('"','')   
    company_links[company_name]=href_link
  #print(company_name,href_link)
  return company_links
#table.find_all('a')

In [ ]:
href_links = get_href_links(soup)
# now adding links in our original df frame
df['company_link'] = df['Security'].apply(lambda x:'https://en.wikipedia.org'+href_links[x])

In [ ]:
df['company_link'].head(2)
# now we have links added in the data frame itself

0                     https://en.wikipedia.org/wiki/3M
1    https://en.wikipedia.org/wiki/Abbott_Laboratories
Name: company_link, dtype: object

In [ ]:
# picking up the text once we have the link
def company_text(company_link):
  """
  The function scrapes wikipedia page for the 
  company given the correct wiki pedia link
  We are scrapping out only the paragraphs 
  element
  """
  # imports
  #import requests as re
  #import bs4 as BeautifulSoup
  # getting page response
  page_response = requests.get(company_link)
  # gettin page text
  page_text = page_response.text
  # getting soup
  soup = bs4.BeautifulSoup(page_text)
  # paragraph_elements
  p_elements = soup.find_all('p')
  # text in paragraphs
  text = [ p.text for p in p_elements if p.text not in ['\n','\xa0']]
  """
  The intention here is to scrap the basic data and not worry about sophisticated
  means of preprocessing. Will handle it during latter part in detailed preprocessing.
  """
  return text

In [ ]:
# finally we are ready to get the text saved in our original data frame
df['company_text'] = df['company_link'].apply(lambda x:company_text(x))  

In [ ]:
# lets save this data frame
df.to_csv('companies_data.csv') # can use index=False # but in excel it is hard to distingush values from one another

In [ ]:
df[df['company_text'].apply(lambda x:len(x))<6].head(1)

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded,company_link,company_text
34,AMT,American Tower Corp.,reports,Real Estate,Specialized REITs,"Boston, Massachusetts",2007-11-19,1053507,1995,https://en.wikipedia.org/wiki/American_Tower_C...,[American Tower Corporation (also referred to ...
